In [1]:
import pandas as pd

bikes=pd.read_csv("bikes.csv", sep=";")
bikeshops=pd.read_csv("bikeshops.csv", sep=";")
orders=pd.read_csv("orders.csv", sep=";")
customers=pd.read_csv("Customers.csv", sep=";")

print(bikes.columns)
print(bikeshops.columns)
print(orders.columns)
print(customers.columns)

Index(['bike_id', 'model', 'category1', 'category2', 'frame', 'price'], dtype='object')
Index(['bikeshop_id', 'bikeshop_name', 'bikeshop_city', 'bikeshop_state',
       'latitude', 'longitude'],
      dtype='object')
Index(['order_id', 'order_line', 'order_date', 'customer_id', 'product_id',
       'bikeshop_id', 'quantity'],
      dtype='object')
Index(['CustomerKey', 'Prefix', 'FirstName', 'LastName', 'BirthDate',
       'MaritalStatus', 'Gender', 'EmailAddress', 'AnnualIncome',
       'TotalChildren', 'EducationLevel', 'Occupation', 'HomeOwner'],
      dtype='object')


In [2]:
bikeshops.head()

,bikeshop_id,bikeshop_name,bikeshop_city,bikeshop_state,latitude,longitude
0,1,Pittsburgh Mountain Machines,Pittsburgh,PA,"40,440625","-79,995886"
1,2,Ithaca Mountain Climbers,Ithaca,NY,"42,443961","-76,501881"
2,3,Columbus Race Equipment,Columbus,OH,"39,961176","-82,998794"
3,4,Detroit Cycles,Detroit,MI,"42,331427","-83,045754"
4,5,Cincinnati Speed,Cincinnati,OH,"39,103118","-84,51202"


In [8]:
bikeshops["latitude"]=bikeshops["latitude"].str.replace(",",".").astype(float)
bikeshops["longitude"]=bikeshops["longitude"].str.replace(",",".").astype(float)

In [10]:
bikeshops.dtypes

bikeshop_id         int64
bikeshop_name      object
bikeshop_city      object
bikeshop_state     object
latitude          float64
longitude         float64
dtype: object

In [11]:
orders.dtypes

order_id        int64
order_line      int64
order_date     object
customer_id     int64
product_id      int64
bikeshop_id     int64
quantity        int64
dtype: object

In [13]:
orders["order_date"]=pd.to_datetime(orders["order_date"])

In [14]:
orders=orders[["order_id", "order_line", "order_date", "customer_id", "product_id","bikeshop_id", "quantity"]]
orders.dtypes

order_id                int64
order_line              int64
order_date     datetime64[ns]
customer_id             int64
product_id              int64
bikeshop_id             int64
quantity                int64
dtype: object

In [15]:
customers.dtypes

CustomerKey        int64
Prefix            object
FirstName         object
LastName          object
BirthDate         object
MaritalStatus     object
Gender            object
EmailAddress      object
AnnualIncome       int64
TotalChildren      int64
EducationLevel    object
Occupation        object
HomeOwner         object
dtype: object

In [16]:
customers["BirthDate"]=pd.to_datetime(customers["BirthDate"], format='mixed', dayfirst=True)
customers=customers[["CustomerKey",	"FirstName", "LastName", "BirthDate", "MaritalStatus", "Gender", "AnnualIncome", "TotalChildren", "EducationLevel", "Occupation", "HomeOwner"]]
customers.dtypes

CustomerKey                int64
FirstName                 object
LastName                  object
BirthDate         datetime64[ns]
MaritalStatus             object
Gender                    object
AnnualIncome               int64
TotalChildren              int64
EducationLevel            object
Occupation                object
HomeOwner                 object
dtype: object

In [17]:
# Supprimer les doublons
bikes.drop_duplicates(inplace=True)
bikeshops.drop_duplicates(inplace=True)
orders.drop_duplicates(inplace=True)
customers.drop_duplicates(inplace=True)

# Supprimer les lignes avec trop de valeurs manquantes (ex: plus de 30% de colonnes vides)
bikes.dropna(thresh=int(0.7 * bikes.shape[1]), inplace=True)
bikeshops.dropna(thresh=int(0.7 * bikeshops.shape[1]), inplace=True)
orders.dropna(thresh=int(0.7 * orders.shape[1]), inplace=True)
customers.dropna(thresh=int(0.7 * customers.shape[1]), inplace=True)


In [18]:
# Joindre les vélos avec les commandes
orders = orders.merge(bikes, left_on="product_id", right_on="bike_id", how="left")

# Joindre les clients aux commandes
orders = orders.merge(customers, left_on="customer_id", right_on="CustomerKey", how="left")

# Joindre les magasins aux commandes
orders = orders.merge(bikeshops, left_on="bikeshop_id", right_on="bikeshop_id", how="left")

# Supprimer ou renommer les colonnes redondantes
orders = orders.drop(columns=["bike_id", "CustomerKey"])  # ou .rename si tu veux garder les données



In [19]:
orders.shape

(15644, 27)

In [20]:
orders.head()

,order_id,order_line,order_date,customer_id,product_id,bikeshop_id,quantity,model,category1,category2,...,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner,bikeshop_name,bikeshop_city,bikeshop_state,latitude,longitude
0,1,1,2011-01-07,2,48,7,1,Jekyll Carbon 2,Mountain,Over Mountain,...,60000,2,Master's,Doctor,No,Nashville Cruisers,Nashville,TN,36.162664,-86.781602
1,1,2,2011-01-07,2,52,26,1,Trigger Carbon 2,Mountain,Over Mountain,...,60000,2,Master's,Doctor,No,Miami Race Equipment,Miami,FL,25.761680,-80.191790
2,2,1,2011-01-10,10,76,17,1,Beast of the East 1,Mountain,Trail,...,46000,1,Associate's,Engineer,Yes,Las Vegas Cycles,Las Vegas,NV,36.169941,-115.139830
3,2,2,2011-01-10,10,52,1,1,Trigger Carbon 2,Mountain,Over Mountain,...,46000,1,Associate's,Engineer,Yes,Pittsburgh Mountain Machines,Pittsburgh,PA,40.440625,-79.995886
4,3,1,2011-01-10,6,2,7,1,Supersix Evo Hi-Mod Team,Road,Elite Road,...,32000,0,Associate's,Student,No,Nashville Cruisers,Nashville,TN,36.162664,-86.781602


In [21]:
orders["ventes"]= orders["price"]*orders["quantity"]
orders["ventes"]

0         6070
1         5970
2         2770
3         5970
4        10660
         ...  
15639     2660
15640     1350
15641     1680
15642     2880
15643     3200
Name: ventes, Length: 15644, dtype: int64

In [22]:
orders.dtypes

order_id                   int64
order_line                 int64
order_date        datetime64[ns]
customer_id                int64
product_id                 int64
bikeshop_id                int64
quantity                   int64
model                     object
category1                 object
category2                 object
frame                     object
price                      int64
FirstName                 object
LastName                  object
BirthDate         datetime64[ns]
MaritalStatus             object
Gender                    object
AnnualIncome               int64
TotalChildren              int64
EducationLevel            object
Occupation                object
HomeOwner                 object
bikeshop_name             object
bikeshop_city             object
bikeshop_state            object
latitude                 float64
longitude                float64
ventes                     int64
dtype: object

In [23]:
# Sauvegarde des données propres
orders.to_csv("orders_cleaned.csv", index=False)


In [35]:
orders["bikeshop_state"].value_counts()

bikeshop_state
TX    1636
FL    1086
NY    1075
PA    1034
CA    1029
MI    1024
OH     998
KS     994
AZ     556
MN     538
KY     533
NM     533
OK     532
RI     529
NV     526
CO     513
LA     507
OR     505
TN     505
IN     496
WA     495
Name: count, dtype: int64